In [1]:
#Constants:
c_train_df_prep_file='train_df_prep_file.pkl'
c_valid_df_prep_file='valid_prep_v1.pkl'
c_bst_file='bst.pkl'
c_accuracy='c_accuracy.csv'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
import gc

In [3]:
metrics = 'auc'

In [4]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 4,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
        'metric':metrics
}
target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour','nip_day_test_hh', 'nip_day_hh',
              'nip_hh_os', 'nip_hh_app', 'nip_hh_dev']
categorical = ['app', 'device', 'os', 'channel', 'hour']

In [5]:
train_df=pd.read_pickle(c_train_df_prep_file)
val_df=pd.read_pickle(c_valid_df_prep_file)

# train_df, val_df = train_test_split( train_df_prep, train_size=.95, random_state=99, shuffle=False )
 
print(train_df.info())
print(val_df.info())

print("train size: ", len(train_df))
print("valid size: ", len(val_df))
# del train_df_prep
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122071523 entries, 0 to 122071522
Data columns (total 13 columns):
app                int64
device             int64
os                 int64
channel            int64
attributed_time    object
is_attributed      int64
click_date         object
hour               uint8
nip_day_test_hh    uint32
nip_day_hh         uint16
nip_hh_os          uint16
nip_hh_app         uint16
nip_hh_dev         uint32
dtypes: int64(5), object(2), uint16(3), uint32(2), uint8(1)
memory usage: 9.0+ GB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37168192 entries, 0 to 37168191
Data columns (total 13 columns):
app                int64
device             int64
os                 int64
channel            int64
attributed_time    datetime64[ns]
is_attributed      int64
click_date         object
hour               uint8
nip_day_test_hh    uint32
nip_day_hh         uint16
nip_hh_os          uint16
nip_hh_app         uint16
nip_hh_dev         uint32
dtypes: d

34

In [8]:
gc.collect()

0

In [9]:
VALIDATE = True
MAX_ROUNDS = 500
EARLY_STOP = 50
OPT_ROUNDS = 500
print("Training...")

num_boost_round=MAX_ROUNDS
early_stopping_rounds=EARLY_STOP

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del train_df
gc.collect()
print('done...')

Training...
done...


In [10]:
xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del val_df
gc.collect()

158

In [11]:
lgb_params

{'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.1,
 'num_leaves': 7,
 'max_depth': 4,
 'min_child_samples': 100,
 'max_bin': 100,
 'subsample': 0.7,
 'subsample_freq': 1,
 'colsample_bytree': 0.7,
 'min_child_weight': 0,
 'min_split_gain': 0,
 'nthread': 4,
 'verbose': 0,
 'scale_pos_weight': 99.7}

In [ ]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain, xgvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration
bst.save_model(c_bst_file)
print("\nModel Report")
print("n_estimators : ", n_estimators)
# print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
del xgvalid

/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.957947	valid's auc: 0.951016
[20]	train's auc: 0.965335	valid's auc: 0.957786
[30]	train's auc: 0.967819	valid's auc: 0.959886
[40]	train's auc: 0.969777	valid's auc: 0.962326
[50]	train's auc: 0.971599	valid's auc: 0.964629
[60]	train's auc: 0.972872	valid's auc: 0.965626
[70]	train's auc: 0.973627	valid's auc: 0.966425
[80]	train's auc: 0.974225	valid's auc: 0.967063
[90]	train's auc: 0.97465	valid's auc: 0.967516
[100]	train's auc: 0.974983	valid's auc: 0.967896
[110]	train's auc: 0.975306	valid's auc: 0.968175
[120]	train's auc: 0.975561	valid's auc: 0.96842
[130]	train's auc: 0.97574	valid's auc: 0.968627
[140]	train's auc: 0.975958	valid's auc: 0.968827
[150]	train's auc: 0.976179	valid's auc: 0.969045
[160]	train's auc: 0.976363	valid's auc: 0.969134
[170]	train's auc: 0.976509	valid's auc: 0.969269
[180]	train's auc: 0.976633	valid's auc: 0.969383
[190]	train's auc: 0.976751	valid's auc: 0.969479


In [ ]:
print('l')

In [21]:
evals_results['valid'][metrics][n_estimators-1]

0.96774997679886854

In [22]:

import datetime
date_time=datetime.datetime.now()
train_acc=evals_results['train']['auc'][n_estimators-1]
valid_acc=evals_results['valid']['auc'][n_estimators-1]
bst_iter=n_estimators
comment='Train data 06 14:32:21 -> 08 16 valid:09 04:00 -> 09 15:00'

In [ ]:
x0: 2017-11-06 14:32:21 -> 2017-11-08 16:00:00
x1: 2017-11-09 04:00:00 -> 2017-11-09 15:00:00

In [24]:
#prepare metrics and see
col_list=['date_time','train_acc','valid_acc','best_iter','comment']
curr_data=pd.DataFrame([[date_time,train_acc,valid_acc,bst_iter,comment]], columns=col_list)
curr_data.head()

,date_time,train_acc,valid_acc,best_iter,comment
0,2018-04-09 13:53:44.416075,0.985568,0.96775,61,"Trained with all data, 5 features from andys"


In [25]:
ADD to records
add_to_records=False
log=pd.read_csv(c_accuracy)
log.head()
if(add_to_records):
    log.append(curr_data)
    print('records updated')
    log.head()
    log.to_csv(c_accuracy,index=False)
gc.collect()
#resetCSV
# curr_data.to_csv(c_accuracy,index=False)
# curr_data.head()

,date_time,train_acc,valid_acc,best_iter,comment
0,2018-04-09 13:53:44.416075,0.985568,0.96775,61,"Trained with all data, 5 features from andys"
